In [1]:
import refinitiv.data as rd
import pandas as pd
from DataQuery_utils import *
rd.open_session()

<refinitiv.data.session.Definition object at 0x158475bb0 {name='workspace'}>

We get all global vars and methods from dataquery into this namespace, so we do not need to write code like dataquery.SCREEN, etc.

### Ways to define the screener query

In [2]:
# at class initiation
my_screen = screener(FORMULA('TR.CompanyMarketCap', '>20000000000'),
                     IN('TR.TRBCEconomicSector', 'Technology'),
                     universe='0#.SPX',
                     currency='USD')
my_screen.query

'SCREEN(U(IN(0#.SPX)),TR.CompanyMarketCap>20000000000,IN(TR.TRBCEconomicSector,Technology),CURN=USD)'

In [3]:
# via the express builder
query = SCREEN.express.universe('0#.SPX').conditions(FORMULA('TR.CompanyMarketCap', '>20000000000'), IN(
    'TR.TRBCEconomicSector', 'Technology')).currency('USD').query
query

'SCREEN(U(IN(0#.SPX)),TR.CompanyMarketCap>20000000000,IN(TR.TRBCEconomicSector,Technology),CURN=USD)'

In [4]:
# by setting the parameters one by one
SCREEN.universe = '0#.SPX'
SCREEN.conditions = FORMULA('TR.CompanyMarketCap', '>20000000000'), IN(
    'TR.TRBCEconomicSector', 'Technology')
SCREEN.currency = 'USD'
query = SCREEN.query
query

'SCREEN(U(IN(0#.SPX)),TR.CompanyMarketCap>20000000000,IN(TR.TRBCEconomicSector,Technology),CURN=USD)'

Another example.
The below cell shows, how the user can create the screener query in 1 line of code via the "express" class:

In [5]:
screen = SCREEN.express.universe(Equity(active=True, public=True, primary=True)) \
    .conditions(IN('TR.CoRTradingCountryCode', 'US'), TOP('TR.CompanyMarketCap', 10, nnumber)) \
    .currency('USD').query

screen

'SCREEN(U(IN(Equity(primary,public,active))),IN(TR.CoRTradingCountryCode,US),TOP(TR.CompanyMarketCap,10,nnumber),CURN=USD)'

The user can also get & set the parameters like universe, conditions, currency by setting them as properties directly in the SCREEN class:

In [6]:
SCREEN.universe


'Equity(primary,public,active)'

In [7]:
SCREEN.query


'SCREEN(U(IN(Equity(primary,public,active))),IN(TR.CoRTradingCountryCode,US),TOP(TR.CompanyMarketCap,10,nnumber),CURN=USD)'

<br>
<br>
We can now use the SCREEN statement in the get_data method with Refinitiv Data Libraries:

In [8]:
df = rd.get_data(screen, ['TR.CompanyMarketCap'])
df


,Instrument,Company Market Cap
0,AAPL.OQ,2433889953180.0
1,MSFT.OQ,1912089813521.709961
2,GOOGL.OQ,1221015510000.0
3,AMZN.OQ,960680602218.75
4,BRKa.N,696137573548.709961
5,TSLA.OQ,613234744480.810059
6,NVDA.OQ,581783800000.0
7,META.OQ,479379052425.200012
8,V.N,466697776273.75
9,XOM.N,463318801484.280029


Here is another query example:

In [9]:
screen2 = SCREEN.express \
    .universe('0#.SPX') \
    .conditions(IN('TR.TRBCEconomicSector', 'Technology')) \
    .currency('USD').query

Let's add the ADC function "Rel" to pull the relative performance of the filtered S&P 500 stocks vs Apple:

In [10]:
df2 = rd.get_data(screen2, ['TR.EpsSmartEst', REL('AAPL.O', 'TR.EpsSmartEst')])
df2

,Instrument,Earnings Per Share - SmartEstimate®,"REL(AAPL.O,TR.EPSSMARTEST)"
0,QRVO.OQ,5.76662,0.964266
1,ON.OQ,4.41964,0.739031
2,AMD.OQ,3.0278,0.506294
3,WDC.OQ,-2.70744,-0.452725
4,ZBRA.OQ,17.29894,2.892645
...,...,...,...
81,FIS.N,5.85639,0.979277
82,ACN.N,11.48844,1.921041
83,LUMN.N,0.31255,0.052263
84,DXC.N,3.48237,0.582305


### We can also specify the screener to look into M&A deals:

In [11]:
screen3 = SCREEN.express.universe(DEALS).conditions(
    IN('TR.MnAStatus', 'C'), RELATIVEDATE('TR.MnARelatedAnnDate', 'LM')).currency('USD').query

In [12]:
df3 = rd.get_data(screen3, ['TR.MNASDCDealNumber',
                  'TR.MnARelatedAcquiror', 'TR.MnADealValue'])
df3

,Instrument,SDC Deal No,Related M&A Deal Acquiror,Deal Value
0,154085483399,3455412040,Inflexion Buyout Fund VI,<NA>
1,154085646481,3541176040,Investcorp Technology Partners,<NA>
2,154087873984,4025299040,Globus Zao,3700000000
3,154087874004,4025311040,Globus Zao,3700000000
4,154087884443,4029433040,Hedin Mobility Group AB,<NA>
...,...,...,...,...
125,154087923140,4047195040,Priner Servicos Industriais SA,<NA>
126,154087924294,4047778020,Envoy Solutions LLC,<NA>
127,154087924303,4047781020,Envoy Solutions LLC,<NA>
128,154087924836,4047994040,Hedin Parts And Logistics AB,<NA>


The rest of this file contains more examples:

In [13]:
screen4 = SCREEN.express.universe('0#.SPX').conditions(
    TOP('TR.CompanyMarketCap', 10, nnumber)).currency('USD').query

In [14]:
df4 = rd.get_data(screen4, ['TR.PriceClose', ROUND(
    FORMULA('TR.PriceClose', '*2'), 0)])
df4

,Instrument,Price Close,"ROUND(TR.PriceClose*2,0)"
0,AAPL.OQ,153.83,308
1,MSFT.OQ,256.87,514
2,GOOGL.OQ,95.13,190
3,GOOG.OQ,95.58,191
4,AMZN.OQ,93.75,188
5,BRKb.N,316.97,634
6,TSLA.OQ,193.81,388
7,NVDA.OQ,235.54,471
8,META.OQ,184.9,370
9,V.N,226.75,454


In [15]:
df5 = rd.get_data('0#.SPX', ['TR.EPSMean', ZSCORE('TR.Close', '1M')])
df5


,Instrument,Earnings Per Share - Mean,SUM(TR.CLOSE-AVG(TR.CLOSE(EDATE=-1M)))/STD(TR.Close(edate=-1M))
0,POOL.OQ,16.36111,-0.885673
1,CHRW.OQ,4.65696,-0.814912
2,AJG.N,8.84881,0.032977
3,CNP.N,1.49164,-0.633258
4,AMCR.N,0.77488,-1.100408
...,...,...,...
498,DXC.N,3.48162,-0.112434
499,SNPS.OQ,10.57683,0.847993
500,J.N,7.36818,0.000648
501,SIVB.OQ,19.26624,-1.065819


In [16]:
SCREEN.express.universe('0#.SPX').conditions(['TR.CompanyMarketCap', '>20000000000'], [
    'TR.TRBCEconomicSector', '=Technology']).currency('USD').query
SCREEN.query

'SCREEN(U(IN(0#.SPX)),TR.CompanyMarketCap>20000000000,TR.TRBCEconomicSector=Technology,CURN=USD)'

In [17]:
df = rd.get_data('0#.SPX', ['TR.TRBCEconomicSector',
                 AVG('TR.Close(Sdate=-30d, Edate=0)')])
df

,Instrument,TRBC Economic Sector Name,"AVG(TR.Close(Sdate=-30d, Edate=0))"
0,POOL.OQ,Consumer Cyclicals,372.611613
1,CHRW.OQ,Industrials,100.943548
2,AJG.N,Financials,191.872581
3,CNP.N,Utilities,29.19
4,AMCR.N,Basic Materials,11.55
...,...,...,...
498,DXC.N,Technology,28.501613
499,SNPS.OQ,Technology,360.335161
500,J.N,Industrials,121.959355
501,SIVB.OQ,Financials,300.137742


In [18]:
df = rd.get_data('0#.SPX', ['TR.TRBCEconomicSector',
                 GAVG('TR.TRBCEconomicSector', 'TR.Close')])
df

,Instrument,TRBC Economic Sector Name,"GAVG(TR.TRBCECONOMICSECTOR,TR.Close)"
0,POOL.OQ,Consumer Cyclicals,6693.358972
1,CHRW.OQ,Industrials,2884.31618
2,AJG.N,Financials,2913.686148
3,CNP.N,Utilities,3380.297974
4,AMCR.N,Basic Materials,2497.731143
...,...,...,...
498,DXC.N,Technology,2021.403997
499,SNPS.OQ,Technology,2021.403997
500,J.N,Industrials,2884.31618
501,SIVB.OQ,Financials,2913.686148


In [19]:
df = rd.get_data('0#.SPX', ['TR.GICSSector', 'TR.EPSMean',
                            GAVG('TR.GICSSector', AVG('TR.EPSMean(SDate=0,EDate=-3,Period=FY1,Frq=Q)'))])
df

,Instrument,GICS Sector Name,Earnings Per Share - Mean,"GAVG(TR.GICSSECTOR,AVG(TR.EPSMean(SDate=0,EDate=-3,Period=FY1,Frq=Q)))"
0,POOL.OQ,Consumer Discretionary,16.36111,195.244695
1,CHRW.OQ,Industrials,4.65696,217.90776
2,AJG.N,Financials,8.84881,169.556108
3,CNP.N,Utilities,1.49164,25.852682
4,AMCR.N,Materials,0.77488,495.59185
...,...,...,...,...
498,DXC.N,Information Technology,3.48162,197.825188
499,SNPS.OQ,Information Technology,10.57683,197.825188
500,J.N,Industrials,7.36818,217.90776
501,SIVB.OQ,Financials,19.26624,169.556108


In [20]:
df = rd.get_data('0#.SPX', AVG(
    'TR.EPSMean(SDate=0,EDate=-3,Period=FY1,Frq=Q)'))
df

,Instrument,"AVG(TR.EPSMean(SDate=0,EDate=-3,Period=FY1,Frq=Q))"
0,POOL.OQ,18.528225
1,CHRW.OQ,7.625805
2,AJG.N,7.622607
3,CNP.N,1.377997
4,AMCR.N,0.800285
...,...,...
498,DXC.N,3.661598
499,SNPS.OQ,8.91802
500,J.N,7.138735
501,SIVB.OQ,30.575745


In [21]:
rd.close_session()